In [13]:
import cv2
import mediapipe as mp

In [14]:
import math

In [15]:
from google.protobuf.json_format import MessageToDict

In [16]:
mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.75,
    min_tracking_confidence=0.75,
    max_num_hands=2)

In [17]:
mpDraw = mp.solutions.drawing_utils

In [21]:
cap = cv2.VideoCapture(0)

while True:

    ret, image = cap.read()

    if not ret:
        break

    mirrored_image = cv2.flip(image, 1)
    
    imgRGB = cv2.cvtColor(mirrored_image, cv2.COLOR_BGR2RGB)
    
    results = hands.process(imgRGB)
    
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            fingerBases = []
            fingerAverage = []
            handBase = []
            distanceBases = []
        
            for id, lm in enumerate(handLms.landmark):
                h, w, c = mirrored_image.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                
                if ((id == 5) or (id == 9)) or ((id == 13) or (id == 17)):
                    fingerBases.append([cx, cy])
                    
                    if (id == 5) or (id == 17):
                        distanceBases.append([cx, cy])
                    
                    #cv2.circle(mirrored_image, (cx, cy), 25, (255, 0, 255), cv2.FILLED)
                    
                elif id == 0:
                    handBase.append([cx, cy])
            
            xAvg = 0
            yAvg = 0
            for i in range(0, len(fingerBases)):
                xAvg += fingerBases[i][0]
                yAvg += fingerBases[i][1]
                
            if len(distanceBases) == 2:
                length = (math.sqrt((distanceBases[0][0] - distanceBases[1][0]) ** 2 + (distanceBases[0][1] - distanceBases[1][1]) ** 2))
                distance = 38.2779 * (0.990764 ** length)
            
            fingerAverage.append([xAvg / len(fingerBases), yAvg / len(fingerBases)])
            
            #print(distance)
            
            for i in range(0, len(fingerAverage)):
                cv2.circle(mirrored_image, (int((fingerAverage[i][0] + handBase[i][0]) / 2), int((fingerAverage[i][1] + handBase[i][1]) / 2)), int(distance), (255, 0, 255), cv2.FILLED)
            
            mpDraw.draw_landmarks(mirrored_image, handLms, mpHands.HAND_CONNECTIONS)
            
        
    
    cv2.imshow("Image", mirrored_image)
    key = cv2.waitKey(1)
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

In [3]:
import cv2
import numpy as np

In [31]:
cap = cv2.VideoCapture(0)

while True:

    ret, image = cap.read()

    if not ret:
        break

    mirrored_image = cv2.flip(image, 1)
    
    gray = cv2.cvtColor(mirrored_image, cv2.COLOR_BGR2GRAY)

    # Find Canny edges
    edged = cv2.Canny(gray, 100, 110)

    # Finding Contours
    # Use a copy of the image e.g. edged.copy()
    # since findContours alters the image
    contours, hierarchy = cv2.findContours(edged, 
        cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    cv2.imshow('Canny Edges After Contouring', edged)

    #print("Number of Contours found = " + str(len(contours)))

    # Draw all contours
    # -1 signifies drawing all contours
    cv2.drawContours(mirrored_image, contours, -1, (0, 255, 0), 3)

    #cv2.imshow("Image", mirrored_image)
    key = cv2.waitKey(1)
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()